# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
import requests
import json
import matplotlib.pyplot as plt
import pandas as pd
from citipy import citipy
# import citipy
import numpy as np
from config import api_key
from pprint import pprint

## Generate Cities List

In [6]:
# Get random numbers for latitude and longitude then place in a dataframe

city_df=pd.DataFrame({"long":(np.random.uniform(-180,180,5)), "lat":(np.random.uniform(-90,90,5))})
# city_df.head()
# city_df.tail()
city_df

,long,lat
0,-48.509881,79.025325
1,-63.115715,-16.853464
2,-106.937886,-0.390971
3,48.474671,-30.035767
4,-176.230789,35.746495


# Use Citipy to Retrieve City Names

In [7]:
# Create lists to store city and country names
# Source: https://github.com/wingchen/citipy
citi=[]
country=[]

for index, row in city_df.iterrows():
    latitude=row['lat']
    longitude=row['long']
    city=citipy.nearest_city(latitude,longitude)
    cityname=city.city_name
    citi.append(cityname)
    cntryname=city.country_code
    country.append(cntryname)
print(citi)
print(country)

# Adds new lists to dataframe
city_df['City']=citi
city_df['Country']=country
city_df

['upernavik', 'mineros', 'puerto ayora', 'taolanaro', 'kapaa']
['gl', 'bo', 'ec', 'mg', 'us']


,long,lat,City,Country
0,-48.509881,79.025325,upernavik,gl
1,-63.115715,-16.853464,mineros,bo
2,-106.937886,-0.390971,puerto ayora,ec
3,48.474671,-30.035767,taolanaro,mg
4,-176.230789,35.746495,kapaa,us


In [26]:
# https://api.openweathermap.org/data/2.5/weather?q=London,uk&appid=api_key&units=imperial
# https://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=api_key&q=London,uk

url = "http://api.openweathermap.org/data/2.5/weather?"
units = 'imperial'
query_url=f"{url}&units={units}&appid={api_key}&q="
# query_url = f"{url}appid={api_key}&units={units}&q="

cityid = []
long = []
lat = []
temp = []
humidity = []
clouds = []
wind = []
maxtemp = []
date = []


# # Build query URL

for index, row in city_df.iterrows():
    cityposition=row['City']
    cntryposition= row['Country']
    response=requests.get(query_url + cityposition +","+ cntryposition).json()
    # pprint(response)
    
    try:
        cityid.append(response['id'])
        long.append(response['coord']['lon'])
        lat.append(response['coord']['lat'])
        temp.append(response['main']['temp'])
        humidity.append(response['main']['humidity'])
        clouds.append(response['clouds']['all'])
        wind.append(response['wind']['speed'])
        maxtemp.append(response['main']['temp_max'])
        date.append(response['dt'])
        print(f"The ID of the city is: {cityid}")
        print(f"The latitude information received is: {lat}")
        print(f"The longitude information received is: {long}")
        print(f"The temperature information received is: {temp}")
        print(f"The humidity information received is: {humidity}")
        print(f"The cloud information received is: {clouds}")
        print(f"The wind speed information received is: {wind}")
        print(f"The maximum temperature received is: {maxtemp}")
        print(f"The date received is: {date}")
        print(f"                                                ")
    
    except:

        print(f"The city can't be found")
        pass
    
    
# url = "http://api.openweathermap.org/data/2.5/weather?"
# units = "imperial"

# # # Build partial query URL
# query_url = f"{url}appid={api_key}&units={units}&q="

#     country=city.country_code
#     country.append(country)
# print(citi)
# print(country)
# cities = ["Paris", "London", "Oslo", "Beijing"]

# url="http://api.openweathermap.org/data/2.5/weather?lat={87.6298}&lon={41.8781}&appid={api_key}"
# # url="http://api.openweathermap.org/data/2.5/weather?q={Chicago}&appid={api_key}?"
# citydata = requests.get(url).json()
# print(json.dumps(citydata, indent=4, sort_keys=True))

# Print the json
# print(geo_data)
# response=requests.get(f"{query_url}'lat='{87.6298}'&lon='{41.8781}").json()
# print(response)

# # set up lists to hold reponse info
# citi = []
# cityid = []
# long = []
# lat = []
# temp = []
# humidity = []
# clouds = []
# wind = []
# country = []

# for index, row in city_df.iterrows():
#     latitude=row['lat']
#     longitude=row['long']
#     response=requests.get(f"{query_url}'lat='{latitude}'&lon='{longitude}").json()
#     print(response)
#     citi.append(response['name'])
#     cityid.append(response['sys']['id'])
#     long.append(response['coord']['lon'])
#     lat.append(response['coord']['lat'])
#     temp.append(response['main']['temp'])
#     humidity.append(response['main']['humidity'])
#     clouds.append(response['clouds']['all'])
#     wind.append(response['wind']['speed'])
#     country.append(response['sys']['country'])

# Loop through the list of cities and perform a request for data on each
# api.openweathermap.org/data/2.5/find?q=London&units=imperial
# for city in cities:
    
#     response = requests.get(query_url + city).json()
#     citi.append(response['name'])
#     cityid.append(response['sys']['id'])
#     long.append(response['coord']['lon'])
#     lat.append(response['coord']['lat'])
#     temp.append(response['main']['temp'])
#     humidity.append(response['main']['humidity'])
#     clouds.append(response['clouds']['all'])
#     wind.append(response['wind']['speed'])
#     country.append(response['sys']['country'])
    
# print(f"The ID of the city is: {cityid}")
# print(f"The city information received is: {citi}")
# print(f"The country information received is: {country}")
# print(f"The latitude information received is: {lat}")
# print(f"The longitude information received is: {long}")
# print(f"The temperature information received is: {temp}")
# print(f"The humidity information received is: {humidity}")
# print(f"The cloud information received is: {clouds}")
# print(f"The wind speed information received is: {wind}")



# # Save config information
# url = "http://api.openweathermap.org/data/2.5/weather?"
# city = "Chicago"

# # Build query URL
# query_url = url + "appid=" + api_key + "&q=" + city

# # Get weather data
# weather_response = requests.get(query_url)
# weather_json = weather_response.json()

# # Get the temperature from the response
# print(f"The weather API responded with: {weather_json}.")
# pprint(weather_json)

The ID of the city is: [3418910]
The latitude information received is: [72.79]
The longitude information received is: [-56.15]
The temperature information received is: [33.58]
The humidity information received is: [92]
The cloud information received is: [71]
The wind speed information received is: [3.31]
The maximum temperature received is: [33.58]
The date received is: [1602512952]
                                                
The ID of the city is: [3418910, 3910343]
The latitude information received is: [72.79, -17.12]
The longitude information received is: [-56.15, -63.23]
The temperature information received is: [33.58, 89.6]
The humidity information received is: [92, 59]
The cloud information received is: [71, 40]
The wind speed information received is: [3.31, 16.11]
The maximum temperature received is: [33.58, 89.6]
The date received is: [1602512952, 1602513165]
                                                
The ID of the city is: [3418910, 3910343, 3652764]
The latitude in

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression